In [1]:
import numpy as np
import pandas as pd
import cv2
import PIL
import pytesseract

image = cv2.imread("ic/Aiman v2.jpg")

def base_All(image):
    x = 15
    y = 120
    w = 960
    h = 485
    cv2.rectangle(image, (x,y),(x+w, y+h), (36,255,12),1)
    cropped_image = image[y:y+h, x:x+w]
    cv2.imwrite('contour.png', cropped_image )
    return image

cropped_image = cv2.imread("contour.png")
cv2.imwrite("temp/all.png",base_All(image))

gray = cv2.cvtColor(cropped_image, cv2.COLOR_BGR2GRAY)
cv2.imwrite("temp/gray.jpg", gray)

blur = cv2.GaussianBlur(gray, (7,7),0)
cv2.imwrite("temp/blur.jpg", blur)

thresh, im_bw = cv2.threshold(gray, 150, 250, cv2.THRESH_BINARY)
cv2.imwrite("temp/thresh.jpg", thresh)
cv2.imwrite("temp/bw.jpg", im_bw)

inverted = cv2.bitwise_not(im_bw)
cv2.imwrite("temp/inverted.jpg", inverted)

def thin_font(image):
    import numpy as np
    image = cv2.bitwise_not(image)
    kernel = np.ones((3,2), np.uint8)
    image = cv2.erode(image, kernel, iterations=1)
    image = cv2.bitwise_not(image)
    return(image)

eroded_image = thin_font(im_bw)
cv2.imwrite("temp/eroded_image.jpg", eroded_image)

def thick_font(image):
    import numpy as np
    image = cv2.bitwise_not(image)
    kernel = np.ones((3,2), np.uint8)
    image = cv2.dilate(image, kernel, iterations=1)
    image = cv2.bitwise_not(image)
    return(image)

dilated_image = thick_font(eroded_image)
cv2.imwrite("temp/dilated_image.jpg", dilated_image)

def noise_removal(image):
    import numpy as np
    kernel = np.ones((1,1), np.uint8)
    image = cv2.dilate(image, kernel, iterations=2)
    kernel = np.ones((1,1), np.uint8)
    image = cv2.erode(image, kernel, iterations=1)
    image = cv2.morphologyEx(image, cv2.MORPH_CLOSE, kernel)
    image = cv2.medianBlur(image, 3)
    return(image)

no_noise = noise_removal(dilated_image)
cv2.imwrite("temp/no_noise.jpg", no_noise)

final_image = thin_font(no_noise)
cv2.imwrite("temp/final_image.jpg", final_image)

#text_cv = pytesseract.image_to_string(thin_font2)
#print(text_cv)

data = pytesseract.image_to_data(final_image)
dataList = list(map(lambda x: x.split('\t'),data.split('\n')))
df = pd.DataFrame(dataList[1:],columns=dataList[0])
df.head(10)




,level,page_num,block_num,par_num,line_num,word_num,left,top,width,height,conf,text
0,1,1,0,0,0,0,0,0,960,485,-1,
1,2,1,1,0,0,0,15,10,544,31,-1,
2,3,1,1,1,0,0,15,10,544,31,-1,
3,4,1,1,1,1,0,15,10,544,31,-1,
4,5,1,1,1,1,1,15,10,308,31,37,980930-01-690.:
5,5,1,1,1,1,2,538,24,21,6,21,=
6,2,1,2,0,0,0,15,278,503,24,-1,
7,3,1,2,1,0,0,15,278,503,24,-1,
8,4,1,2,1,1,0,15,278,503,24,-1,
9,5,1,2,1,1,1,15,279,198,23,57,MUHAMMAI)


In [2]:
df.dropna(inplace=True) # drop the missing in rows
col_int = ['level','page_num','block_num','par_num','line_num','word_num','left','top','width','height','conf']
df[col_int] = df[col_int].astype(int)

In [3]:
cropped = cv2.imread("temp/result.png")
image2 = cropped.copy()

In [4]:
level = 'word'
for l,x,y,w,h,c,txt in df[['level','left','top','width','height','conf','text']].values:
    #print(l,x,y,w,h,c)
    if level == 'page':
        if l == 1:
            cv2.rectangle(image2,(x,y),(x+w,y+h),(0,0,0),2)
        else:
            continue
            
    elif level == 'block':
        if l == 2:
            cv2.rectangle(image2,(x,y),(x+w,y+h),(255,0,0),2)
        else:
            continue
    
    elif level == 'para':
        if l == 3:
            cv2.rectangle(image2,(x,y),(x+w,y+h),(0,255,0),2)
        else:
            continue
    
    elif level == 'line':
        if l == 4:
            cv2.rectangle(image2,(x,y),(x+w,y+h),(0,0,255),2)
        else:
            continue
            
    elif level == 'word':
        if l == 5:
            cv2.rectangle(image2,(x,y),(x+w,y+h),(0,255,0),2)
            cv2.putText(image2,txt,(x,y),cv2.FONT_HERSHEY_PLAIN,1,(255,0,0),2)
        else:
            continue
            
cv2.imshow("bounding box",image2)
cv2.waitKey()
cv2.destroyAllWindows()